In [1]:
import pandas as pd
import os

# 一.係數處理(單一球季RE24)

In [2]:
excel_file_path = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2023.xlsx'  # 請替換成你的 Excel 檔案路徑
sheet_name1 = 'AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 = 'HOME-打席資訊'

In [3]:
away = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
home = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

In [4]:
cols = ['開始出局數','結束出局數','打席結果','落點1','落點2','彈道','RE','RE24']

In [5]:
away_play = away[cols]
home_play = home[cols]

In [6]:
summary = pd.concat([home_play,away_play], ignore_index=True)

In [7]:
summary['出局數']=summary['結束出局數']-summary['開始出局數']

In [8]:
index_to_change = summary[(summary['出局數'] == 2) | (summary['出局數'] == 3)].index

# 將出局數為2的資料改成1
summary.loc[index_to_change, '出局數'] = 1

In [9]:
summary1 = summary.groupby("彈道").agg({"RE": "mean", "RE24": "mean"})
summary1

,RE,RE24
彈道,,
3,1.443000,0.757000
F,0.562248,0.037584
G,0.587815,-0.084250
L,0.568928,0.331434
P,0.562649,-0.310442
x,0.901105,0.260474


In [10]:
summary2 = summary.groupby("打席結果").agg({"RE": "mean", "RE24": "mean"})
summary2

,RE,RE24
打席結果,,
1B,0.567685,0.492290
2B,0.580716,0.803669
3B,0.579073,1.223866
E,0.495731,0.434309
FC,0.852640,-0.309473
FO,0.527666,-0.309818
GIDP,0.995886,-0.888690
GIDP_E,1.066000,-0.680000
GO,0.466751,-0.255090


In [11]:
summary2['RE24']['uBB']

0.34843181818181823

In [12]:
def classify_fly_ball(row):
    #if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        #return '外野飛球'
    if row['彈道'] == 'P'and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    #elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        #return '內野飛球'
    elif row['彈道'] == 'F'and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

summary['飛球分類'] = summary.apply(classify_fly_ball, axis=1)

# 打印结果
summary

,開始出局數,結束出局數,打席結果,落點1,落點2,彈道,RE,RE24,出局數,飛球分類
0,0,1,GO,4.0,4.0,G,0.557,-0.263,1,滾地球
1,1,2,FO,8.0,8.0,F,0.294,-0.179,1,外野飛球
2,2,3,GO,5.0,5.0,G,0.115,-0.115,1,滾地球
3,0,1,FO,9.0,9.0,F,0.557,-0.263,1,外野飛球
4,1,2,SO,NaN,NaN,NaN,0.294,-0.179,1,其他
...,...,...,...,...,...,...,...,...,...,...
22919,0,0,1B,6.0,6.0,G,0.587,0.415,0,滾地球
22920,0,0,uBB,NaN,NaN,NaN,1.002,0.653,0,其他
22921,0,1,SO,NaN,NaN,NaN,1.655,-0.591,1,其他
22922,1,2,SO,NaN,NaN,NaN,1.064,-0.529,1,其他


In [13]:
summary3 = summary.groupby(["飛球分類"]).agg({"RE": "mean", "RE24": "mean"})

# 重新命名列名
#hitter_summary.rename(columns={"li值": "平均LI值", "WPA": "總WPA","打序": "PA"}, inplace=True)

# 輸出結果
summary3

,RE,RE24
飛球分類,,
內野飛球,0.562649,-0.310442
其他,0.544474,-0.097702
外野飛球,0.562248,0.037584
平飛球,0.568928,0.331434
滾地球,0.587815,-0.084250


In [14]:
dfs = {}

# 定義你想分組的欄位
group_columns = ['飛球分類']

# 分組和聚合計算出局數的次數和總和
for name, group in summary.groupby(group_columns):
    df1 = group.groupby(group_columns).agg({'出局數':'count'})
    df2 = group.groupby(group_columns).agg({'出局數':'sum'})
    dfs[name] = pd.merge(df1, df2, on=group_columns)

# 合併所有的 DataFrame
result = pd.concat(dfs.values())

result

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18096\4216358398.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in summary.groupby(group_columns):


,出局數_x,出局數_y
飛球分類,,
內野飛球,1223,1201
其他,5974,4028
外野飛球,5549,3943
平飛球,2238,586
滾地球,7940,5654


In [15]:
result = result.rename(columns={'出局數_x': '總數', '出局數_y': '出局數量'})
result

,總數,出局數量
飛球分類,,
內野飛球,1223,1201
其他,5974,4028
外野飛球,5549,3943
平飛球,2238,586
滾地球,7940,5654


In [16]:
result['出局比例']=result['出局數量']/result['總數']

In [17]:
result

,總數,出局數量,出局比例
飛球分類,,,
內野飛球,1223,1201,0.982011
其他,5974,4028,0.674255
外野飛球,5549,3943,0.710578
平飛球,2238,586,0.261841
滾地球,7940,5654,0.712091


# 二.計算各投手[HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA]

In [18]:
cols = ['客場','主場','投手','打席結果','落點1','落點2','彈道']

In [19]:
away_play = away[cols]
home_play = home[cols]

In [20]:
away_play = away_play.rename(columns={'客場': '對手', '主場': '球隊'})
home_play = home_play.rename(columns={'客場': '球隊', '主場': '對手'})

In [21]:
away_play

,對手,球隊,投手,打席結果,落點1,落點2,彈道
0,統一7-ELEVEn獅,富邦悍將,安得勝,1B,5.0,5.0,G
1,統一7-ELEVEn獅,富邦悍將,安得勝,FC,4.0,4.0,G
2,統一7-ELEVEn獅,富邦悍將,安得勝,FO,3.0,3.0,L
3,統一7-ELEVEn獅,富邦悍將,安得勝,GO,6.0,6.0,G
4,統一7-ELEVEn獅,富邦悍將,安得勝,GO,6.0,6.0,G
...,...,...,...,...,...,...,...
11634,富邦悍將,中信兄弟,李振昌,1B,6.0,6.0,G
11635,富邦悍將,中信兄弟,李振昌,uBB,NaN,NaN,NaN
11636,富邦悍將,中信兄弟,李振昌,SO,NaN,NaN,NaN
11637,富邦悍將,中信兄弟,李振昌,SO,NaN,NaN,NaN


In [22]:
summary = pd.concat([home_play,away_play], ignore_index=True)
summary

,球隊,對手,投手,打席結果,落點1,落點2,彈道
0,統一7-ELEVEn獅,富邦悍將,羅昂,GO,4.0,4.0,G
1,統一7-ELEVEn獅,富邦悍將,羅昂,FO,8.0,8.0,F
2,統一7-ELEVEn獅,富邦悍將,羅昂,GO,5.0,5.0,G
3,統一7-ELEVEn獅,富邦悍將,羅昂,FO,9.0,9.0,F
4,統一7-ELEVEn獅,富邦悍將,羅昂,SO,NaN,NaN,NaN
...,...,...,...,...,...,...,...
22919,中信兄弟,富邦悍將,李振昌,1B,6.0,6.0,G
22920,中信兄弟,富邦悍將,李振昌,uBB,NaN,NaN,NaN
22921,中信兄弟,富邦悍將,李振昌,SO,NaN,NaN,NaN
22922,中信兄弟,富邦悍將,李振昌,SO,NaN,NaN,NaN


#### 1. 計算三振,觸身球,保送和全壘打數量

In [23]:
def calculate_so_go_count(series):
    so_count = (series == 'SO').sum()
    ubb_count = (series == 'uBB').sum()
    hbp_count = (series == 'HBP').sum()
    hr_count = (series == 'HR').sum()
    return pd.Series({'SO': so_count,
                      'HR':hr_count
                     , 'uBB':ubb_count
                     , 'HBP':hbp_count})

In [24]:
resault_away=away_play.groupby(['投手','球隊'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_away = resault_away.rename(columns={'球隊':'球隊','level_2': '打席結果', '打席結果': '次數'})

resault_home=home_play.groupby(['投手','球隊'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_home = resault_home.rename(columns={'球隊':'球隊','level_2':'打席結果', '打席結果':'次數'})

merged = pd.merge(resault_away, resault_home, on=['投手', '球隊','打席結果'], how='outer')
merged.fillna(0, inplace=True)
merged['次數']=merged['次數_x']+merged['次數_y']

merged

,投手,球隊,打席結果,次數_x,次數_y,次數
0,伍鐸,味全龍,SO,23.0,20.0,43.0
1,伍鐸,味全龍,HR,1.0,0.0,1.0
2,伍鐸,味全龍,uBB,9.0,7.0,16.0
3,伍鐸,味全龍,HBP,1.0,3.0,4.0
4,余德龍,樂天桃猿,SO,0.0,0.0,0.0
...,...,...,...,...,...,...
547,霸曼,味全龍,HBP,0.0,0.0,0.0
548,高國慶,統一7-ELEVEn獅,SO,0.0,0.0,0.0
549,高國慶,統一7-ELEVEn獅,HR,0.0,0.0,0.0
550,高國慶,統一7-ELEVEn獅,uBB,0.0,1.0,1.0


In [25]:
playresult_items = merged['投手'].unique()
playresult_items

array(['伍鐸', '余德龍', '傅于剛', '優瑪', '克維斯', '劉予承', '劉家愷', '劉昱言', '劉軒荅',
       '古林睿煬', '吳俊偉', '吳俊杰', '吳哲源', '呂偉晟', '呂彥青', '呂詠臻', '周磊', '奧特羅',
       '姚杰宏', '威迪茲', '安得勝', '宋文華', '官大元', '富藍戈', '岳少華', '布里悍', '布雷克',
       '廖乙忠', '廖任磊', '廖文揚', '張梓軒', '張瑞麟', '張耿豪', '張鈞守', '彭識穎', '德保拉',
       '必贏多', '施子謙', '曹祐齊', '曾仁和', '曾峻岳', '朱俊祥', '李其峰', '李吳永勤', '李子強',
       '李建勲', '李振昌', '林丞軒', '林凱威', '林子崴', '林子昱', '林羿豪', '林逸達', '柯瑞',
       '楊孟沅', '楊彬', '楊志龍', '歐書誠', '江國謙', '江國豪', '江少慶', '江忠城', '江承峰',
       '江辰晏', '泰樂', '泰迪', '洪聖欽', '游宗儒', '游霆崴', '潘威倫', '牧田和久', '狂威', '王凱程',
       '王奕凱', '王尉永', '王志煊', '王溢正', '王玉譜', '王維中', '王躍霖', '王鏡銘', '索沙', '羅力',
       '羅國華', '羅昂', '羅華韋', '翁瑋均', '胡智爲', '范玉禹', '范米特', '莊昕諺', '蔡齊哲',
       '藍愷青', '蘇俊璋', '蘇俊羽', '許峻暘', '謝榮豪', '象魔力', '豪勁', '賴智垣', '賴知頎',
       '賴鴻誠', '趙璟榮', '邱浩鈞', '邱駿威', '郭俊麟', '郭郁政', '鄭凱文', '鄭鈞仁', '鋼龍',
       '陳仕朋', '陳克羿', '陳冠偉', '陳冠勳', '陳冠宇', '陳明軒', '陳柏豪', '陳禹勳', '陳韋霖',
       '陳韻文', '陳鴻文', '霸林爵', '魏碩成', '麥力德', '黃偉晟', '黃子鵬', '黃恩賜', '黃東淯',
       '黃竣彥', '宋

In [26]:
merged['次數'] = merged['次數'].astype(int)

merged = pd.pivot_table(merged, index=['投手', '球隊'], columns=['打席結果'], values=['次數'], aggfunc='sum', fill_value=0)

merged

次數           
打席結果            HBP HR  SO uBB
投手  球隊                        
伍鐸  味全龍           4  1  43  16
余德龍 樂天桃猿          0  0   0   1
傅于剛 統一7-ELEVEn獅   1  1   4   7
優瑪  富邦悍將          0  0   4   2
克維斯 統一7-ELEVEn獅   2  3  59  32
...              .. ..  ..  ..
黃偉晟 樂天桃猿          1  1  10   3
黃子鵬 樂天桃猿         10  8  96  25
黃恩賜 中信兄弟          2  1  12   5
黃東淯 味全龍           0  0   1   1
黃竣彥 統一7-ELEVEn獅   2  2  11   6

[138 rows x 4 columns]

#### 2.整理飛球分類 並計算數量

In [27]:
def classify_fly_ball(row):
    #if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        #return '外野飛球'
    if row['彈道'] == 'P' and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    #elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        #return '內野飛球'
    elif row['彈道'] == 'F' and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

summary['飛球分類'] = summary.apply(classify_fly_ball, axis=1)

# 打印结果
summary


,球隊,對手,投手,打席結果,落點1,落點2,彈道,飛球分類
0,統一7-ELEVEn獅,富邦悍將,羅昂,GO,4.0,4.0,G,滾地球
1,統一7-ELEVEn獅,富邦悍將,羅昂,FO,8.0,8.0,F,外野飛球
2,統一7-ELEVEn獅,富邦悍將,羅昂,GO,5.0,5.0,G,滾地球
3,統一7-ELEVEn獅,富邦悍將,羅昂,FO,9.0,9.0,F,外野飛球
4,統一7-ELEVEn獅,富邦悍將,羅昂,SO,NaN,NaN,NaN,其他
...,...,...,...,...,...,...,...,...
22919,中信兄弟,富邦悍將,李振昌,1B,6.0,6.0,G,滾地球
22920,中信兄弟,富邦悍將,李振昌,uBB,NaN,NaN,NaN,其他
22921,中信兄弟,富邦悍將,李振昌,SO,NaN,NaN,NaN,其他
22922,中信兄弟,富邦悍將,李振昌,SO,NaN,NaN,NaN,其他


In [28]:
def calculate_so_go_count(series):
    iff_count = (series == '內野飛球').sum()
    off_count = (series == '外野飛球').sum()
    ld_count = (series == '平飛球').sum()
    gr_count = (series == '滾地球').sum()
    return pd.Series({'內野飛球': iff_count,
                      '外野飛球':off_count
                     , '平飛球':ld_count
                     , '滾地球':gr_count})

In [29]:
resault2=summary.groupby(['投手','球隊'])['飛球分類'].apply(calculate_so_go_count).reset_index()
resault2 = resault2.rename(columns={'level_2': '擊球分類', '飛球分類': '次數'})

resault2['次數'] = resault2['次數'].astype(int)

final_resault2 = pd.pivot_table(resault2, index=['投手','球隊'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault2

,擊球分類,內野飛球,外野飛球,平飛球,滾地球
投手,球隊,,,,
伍鐸,味全龍,10,89,44,136
余德龍,樂天桃猿,0,3,0,0
傅于剛,統一7-ELEVEn獅,4,18,5,14
優瑪,富邦悍將,2,8,2,12
克維斯,統一7-ELEVEn獅,13,54,16,84
...,...,...,...,...,...
黃偉晟,樂天桃猿,6,29,7,32
黃子鵬,樂天桃猿,36,166,70,233
黃恩賜,中信兄弟,6,26,13,54


In [30]:
pitcher = pd.merge(merged, final_resault2,on=['投手','球隊'])
pitcher.rename(columns={('次數', 'HBP'): 'HBP',
                   ('次數', 'HR'): 'HR',
                   ('次數', 'SO'): 'SO',
                   ('次數', 'uBB'): 'uBB'}, inplace=True)
pitcher

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18096\2221179295.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  pitcher = pd.merge(merged, final_resault2,on=['投手','球隊'])


,,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
投手,球隊,,,,,,,,
伍鐸,味全龍,4,1,43,16,10,89,44,136
余德龍,樂天桃猿,0,0,0,1,0,3,0,0
傅于剛,統一7-ELEVEn獅,1,1,4,7,4,18,5,14
優瑪,富邦悍將,0,0,4,2,2,8,2,12
克維斯,統一7-ELEVEn獅,2,3,59,32,13,54,16,84
...,...,...,...,...,...,...,...,...,...
黃偉晟,樂天桃猿,1,1,10,3,6,29,7,32
黃子鵬,樂天桃猿,10,8,96,25,36,166,70,233
黃恩賜,中信兄弟,2,1,12,5,6,26,13,54


In [31]:
pitcher.reset_index(drop=False, inplace=True)
pitcher = pitcher[['投手', '球隊', 'HBP', 'HR', 'SO', 'uBB', '內野飛球', '外野飛球', '平飛球', '滾地球']]
pitcher

,投手,球隊,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
0,伍鐸,味全龍,4,1,43,16,10,89,44,136
1,余德龍,樂天桃猿,0,0,0,1,0,3,0,0
2,傅于剛,統一7-ELEVEn獅,1,1,4,7,4,18,5,14
3,優瑪,富邦悍將,0,0,4,2,2,8,2,12
4,克維斯,統一7-ELEVEn獅,2,3,59,32,13,54,16,84
...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,1,1,10,3,6,29,7,32
134,黃子鵬,樂天桃猿,10,8,96,25,36,166,70,233
135,黃恩賜,中信兄弟,2,1,12,5,6,26,13,54
136,黃東淯,味全龍,0,0,1,1,0,6,1,1


#### 3.計算聯盟定數 
# (聯盟ERA和FIP要自填)

In [32]:
lg_ERA = 3.70  #需填入該年度ERA
lg_FIP = 3.70

In [33]:
uBB = pitcher['uBB'].sum()
HBP = pitcher['HBP'].sum()
SO = pitcher['SO'].sum()
HR = pitcher['HR'].sum()
滾地球 = pitcher['滾地球'].sum()
IFF = pitcher['內野飛球'].sum()
OFF = pitcher['外野飛球'].sum()
平飛球 = pitcher['平飛球'].sum()


constant = (lg_ERA - (
    (summary2['RE24']['uBB'] * uBB 
     + summary2['RE24']['HBP'] * HBP 
     + summary2['RE24']['SO'] * SO 
     + summary2['RE24']['HR'] * HR 
     + summary3['RE24']['滾地球'] * 滾地球 
     + summary3['RE24']['內野飛球'] * IFF
     + summary3['RE24']['外野飛球'] * OFF 
     + summary3['RE24']['平飛球'] * 平飛球)) /
    (SO + result['出局比例']['滾地球'] * 滾地球 + result['出局比例']['平飛球'] * 平飛球 + result['出局比例']['內野飛球'] * IFF + result['出局比例']['外野飛球'] * OFF) * 27
)
constant

3.765291737482024

In [34]:
lg_tRA = (summary2['RE24']['uBB'] * uBB 
     +summary2['RE24']['HBP'] * HBP 
     + summary2['RE24']['SO'] * SO 
     + summary2['RE24']['HR'] * HR 
     + summary3['RE24']['滾地球'] * 滾地球 
     +summary3['RE24']['內野飛球'] * IFF
     +summary3['RE24']['外野飛球'] * OFF 
     + summary3['RE24']['平飛球'] * 平飛球)/((SO + result['出局比例']['滾地球'] * 滾地球 + result['出局比例']['平飛球'] * 平飛球 + result['出局比例']['內野飛球'] * IFF + result['出局比例']['外野飛球'] * OFF) * 27)+constant
                
lg_tRA

3.764717949090416

In [35]:
pitcher['tRA']=((pitcher['uBB']*summary2['RE24']['uBB']
                 +pitcher['HBP']*summary2['RE24']['HBP']
                 +pitcher['SO']*summary2['RE24']['SO']
                 +pitcher['HR']*summary2['RE24']['HR']
                 +pitcher['滾地球']* summary3['RE24']['滾地球']
                 +pitcher['內野飛球']*summary3['RE24']['內野飛球']
                 +pitcher['外野飛球']*summary3['RE24']['外野飛球']
                 +pitcher['平飛球']*summary3['RE24']['平飛球'])/
                (pitcher['SO']
                 +pitcher['滾地球']*result['出局比例']['滾地球']
                +pitcher['平飛球']*result['出局比例']['平飛球']
                +pitcher['內野飛球']*result['出局比例']['內野飛球']
                +pitcher['外野飛球']*result['出局比例']['外野飛球'])*27)+constant

In [36]:
pitcher

,投手,球隊,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
0,伍鐸,味全龍,4,1,43,16,10,89,44,136,3.594014
1,余德龍,樂天桃猿,0,0,0,1,0,3,0,0,9.606541
2,傅于剛,統一7-ELEVEn獅,1,1,4,7,4,18,5,14,6.424623
3,優瑪,富邦悍將,0,0,4,2,2,8,2,12,2.163124
4,克維斯,統一7-ELEVEn獅,2,3,59,32,13,54,16,84,2.931594
...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,1,1,10,3,6,29,7,32,3.239529
134,黃子鵬,樂天桃猿,10,8,96,25,36,166,70,233,3.413535
135,黃恩賜,中信兄弟,2,1,12,5,6,26,13,54,3.512223
136,黃東淯,味全龍,0,0,1,1,0,6,1,1,5.958443


# 三.投手一般成績整理

In [37]:
sheet_name3 = 'AWAY-BOX-投球'  # 請替換成你要載入的工作表名稱
sheet_name4 = 'HOME-BOX-投球'

away1 = pd.read_excel(excel_file_path, sheet_name=sheet_name3)
home1 = pd.read_excel(excel_file_path, sheet_name=sheet_name4)

In [38]:
away1.rename(columns={'客場': '球隊'}, inplace=True)
home1.rename(columns={'主場': '球隊'}, inplace=True)

In [39]:
merged_stat = pd.concat([away1, home1], ignore_index=True)

#merged_stat = merged_stat.drop(["比賽編號","比賽日期","客場","主場","背號"], axis = 1)
merged_stat

,比賽編號,比賽日期,球場,球隊,主場,順序,背號,名稱,出局數,投球數,...,全壘打,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,客場
0,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,1,44,羅昂,6,17,...,0,0,0,0,1,0,0,0,0,NaN
1,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,2,17,林子崴,14,67,...,0,1,0,0,9,0,0,1,1,NaN
2,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,3,45,施子謙,1,5,...,0,0,0,0,0,0,0,0,0,NaN
3,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,4,86,李承鴻,6,19,...,0,0,0,0,1,0,0,0,0,NaN
4,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,5,23,劉軒荅,3,10,...,0,0,0,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,3,2022-04-03 17:06,臺南市立棒球場,統一7-ELEVEn獅,NaN,6,37,邱浩鈞,4,26,...,0,1,0,0,3,0,0,0,0,味全龍
2450,3,2022-04-03 17:06,臺南市立棒球場,統一7-ELEVEn獅,NaN,7,26,劉予承,2,10,...,0,0,0,0,1,0,0,0,0,味全龍
2451,2,2022-04-03 17:05,臺中市洲際棒球場,中信兄弟,NaN,1,55,象魔力,21,89,...,0,2,0,0,7,0,0,0,0,富邦悍將
2452,2,2022-04-03 17:05,臺中市洲際棒球場,中信兄弟,NaN,2,79,謝榮豪,3,12,...,1,0,0,0,1,0,0,1,1,富邦悍將


In [40]:
# 此Park Factor為2023年
park_factors = {
    "臺北市立天母棒球場": 0.974586135,
    "臺南市立棒球場": 1.049876521,
    "新北市立新莊棒球場": 0.982791461,
    "樂天桃園棒球場": 1.096458186,
    "臺中市洲際棒球場":0.909723723,
    "嘉義市立棒球場":0.938006629,
    "斗六棒球場":1.054394354,
    "澄清湖棒球場":0.932267771,
    "花蓮縣立德興棒球場":1.012278069,
    "台東棒球村第一棒球場":0.726667302,
    "新竹市立棒球場":0.298210736
}

merged_stat["PF(tRA)"] = merged_stat["球場"].map(park_factors)

merged_stat

,比賽編號,比賽日期,球場,球隊,主場,順序,背號,名稱,出局數,投球數,...,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,客場,PF(tRA)
0,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,1,44,羅昂,6,17,...,0,0,0,1,0,0,0,0,NaN,0.982791
1,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,2,17,林子崴,14,67,...,1,0,0,9,0,0,1,1,NaN,0.982791
2,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,3,45,施子謙,1,5,...,0,0,0,0,0,0,0,0,NaN,0.982791
3,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,4,86,李承鴻,6,19,...,0,0,0,1,0,0,0,0,NaN,0.982791
4,154,2022-10-26 18:35,新北市立新莊棒球場,統一7-ELEVEn獅,富邦悍將,5,23,劉軒荅,3,10,...,0,0,0,1,0,0,0,0,NaN,0.982791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,3,2022-04-03 17:06,臺南市立棒球場,統一7-ELEVEn獅,NaN,6,37,邱浩鈞,4,26,...,1,0,0,3,0,0,0,0,味全龍,1.049877
2450,3,2022-04-03 17:06,臺南市立棒球場,統一7-ELEVEn獅,NaN,7,26,劉予承,2,10,...,0,0,0,1,0,0,0,0,味全龍,1.049877
2451,2,2022-04-03 17:05,臺中市洲際棒球場,中信兄弟,NaN,1,55,象魔力,21,89,...,2,0,0,7,0,0,0,0,富邦悍將,0.909724
2452,2,2022-04-03 17:05,臺中市洲際棒球場,中信兄弟,NaN,2,79,謝榮豪,3,12,...,0,0,0,1,0,0,1,1,富邦悍將,0.909724


In [41]:
agg_dict = {
    "出局數": "sum",
    "面對打者": "sum",
    "安打": "sum",
    "全壘打": "sum",
    "保送": "sum",
    "故意四壞": "sum",
    "觸身": "sum",
    "三振": "sum",
    "暴投": "sum",
    "投手犯規": "sum",
    "失分": "sum",
    "責失": "sum",
    "PF(tRA)": "mean"
}
pitcher_total = merged_stat.groupby(["名稱",'球隊']).agg(agg_dict).reset_index()
pitcher_total

,名稱,球隊,出局數,面對打者,安打,全壘打,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,PF(tRA)
0,伍鐸,味全龍,249,347,77,1,21,5,4,43,0,0,38,21,0.981843
1,余德龍,樂天桃猿,3,4,0,0,1,0,0,0,0,0,0,0,1.096458
2,傅于剛,統一7-ELEVEn獅,35,53,12,1,7,0,1,4,0,0,10,9,1.002275
3,優瑪,富邦悍將,19,30,8,0,2,0,0,4,0,0,4,4,1.041675
4,克維斯,統一7-ELEVEn獅,183,260,50,3,32,0,2,59,0,0,25,24,1.032662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,64,88,23,1,3,0,1,10,0,0,10,10,1.063268
134,黃子鵬,樂天桃猿,476,636,143,8,25,0,10,96,0,0,45,41,1.033766
135,黃恩賜,中信兄弟,77,118,37,1,5,0,2,12,0,0,15,13,0.978773
136,黃東淯,味全龍,5,10,4,0,1,0,0,1,0,0,1,1,1.041728


In [42]:
pitcher_total["IP"] = pitcher_total["出局數"] / 3
pitcher_total["ERA"] = (pitcher_total["責失"] / pitcher_total["IP"]) * 9
pitcher_total["FIP"] = (13 *pitcher_total["全壘打"] + 3 * pitcher_total["保送"] + 2 * pitcher_total["觸身"] + 3 * pitcher_total["暴投"] - 2 * pitcher_total["三振"]) / pitcher_total["IP"] + 3.2
pitcher_total["K%"] = (pitcher_total["三振"] / pitcher_total["面對打者"]) * 100
pitcher_total["BB%"] = (pitcher_total["保送"] / pitcher_total["面對打者"]) * 100
pitcher_total["ERA+"] = (lg_ERA / pitcher_total["ERA"]) * 100
pitcher_total["FIP+"] = (lg_FIP / pitcher_total["FIP"]) * 100

# 移除指定的欄位
pitcher_total.drop(columns=["出局數","暴投","投手犯規","故意四壞","觸身"], inplace=True)
pitcher_total = pitcher_total.rename(columns={'名稱': '投手'})

# 列出計算結果
pitcher_total

,投手,球隊,面對打者,安打,全壘打,保送,三振,失分,責失,PF(tRA),IP,ERA,FIP,K%,BB%,ERA+,FIP+
0,伍鐸,味全龍,347,77,1,21,43,38,21,0.981843,83.000000,2.277108,3.175904,12.391931,6.051873,146.984656,105.576252
1,余德龍,樂天桃猿,4,0,0,1,0,0,0,1.096458,1.000000,0.000000,6.200000,0.000000,25.000000,inf,54.080645
2,傅于剛,統一7-ELEVEn獅,53,12,1,7,4,10,9,1.002275,11.666667,6.942857,5.600000,7.547170,13.207547,48.207819,59.875000
3,優瑪,富邦悍將,30,8,0,2,4,4,4,1.041675,6.333333,5.684211,2.884211,13.333333,6.666667,58.882407,116.253650
4,克維斯,統一7-ELEVEn獅,260,50,3,32,59,25,24,1.032662,61.000000,3.540984,3.544262,22.692308,12.307692,94.521759,94.603608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,88,23,1,3,10,10,10,1.063268,21.333333,4.218750,3.387500,11.363636,3.409091,79.336296,98.981550
134,黃子鵬,樂天桃猿,636,143,8,25,96,45,41,1.033766,158.666667,2.325630,3.244118,15.094340,3.930818,143.917977,103.356301
135,黃恩賜,中信兄弟,118,37,1,5,12,15,13,0.978773,25.666667,4.558442,3.511688,10.169492,4.237288,73.424217,95.481139
136,黃東淯,味全龍,10,4,0,1,1,1,1,1.041728,1.666667,5.400000,3.800000,10.000000,10.000000,61.981481,88.236842


# 四.投手成績和tRA合併

## 這裡合併完可以檢查前面是否有算錯 EX:兩個檔案的保送或三振數是否一樣

In [43]:
all_D = pd.merge(pitcher_total, pitcher, on=['投手','球隊'], how='inner')
all_D

,投手,球隊,面對打者,安打,全壘打,保送,三振,失分,責失,PF(tRA),...,FIP+,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
0,伍鐸,味全龍,347,77,1,21,43,38,21,0.981843,...,105.576252,4,1,43,16,10,89,44,136,3.594014
1,余德龍,樂天桃猿,4,0,0,1,0,0,0,1.096458,...,54.080645,0,0,0,1,0,3,0,0,9.606541
2,傅于剛,統一7-ELEVEn獅,53,12,1,7,4,10,9,1.002275,...,59.875000,1,1,4,7,4,18,5,14,6.424623
3,優瑪,富邦悍將,30,8,0,2,4,4,4,1.041675,...,116.253650,0,0,4,2,2,8,2,12,2.163124
4,克維斯,統一7-ELEVEn獅,260,50,3,32,59,25,24,1.032662,...,94.603608,2,3,59,32,13,54,16,84,2.931594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,88,23,1,3,10,10,10,1.063268,...,98.981550,1,1,10,3,6,29,7,32,3.239529
134,黃子鵬,樂天桃猿,636,143,8,25,96,45,41,1.033766,...,103.356301,10,8,96,25,36,166,70,233,3.413535
135,黃恩賜,中信兄弟,118,37,1,5,12,15,13,0.978773,...,95.481139,2,1,12,5,6,26,13,54,3.512223
136,黃東淯,味全龍,10,4,0,1,1,1,1,1.041728,...,88.236842,0,0,1,1,0,6,1,1,5.958443


In [44]:
pitcher = all_D[['投手', '球隊','IP','ERA','FIP', 'HBP', 'HR', 'SO', 'uBB', '內野飛球', '外野飛球', '平飛球', '滾地球','tRA','PF(tRA)']]
pitcher

,投手,球隊,IP,ERA,FIP,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA,PF(tRA)
0,伍鐸,味全龍,83.000000,2.277108,3.175904,4,1,43,16,10,89,44,136,3.594014,0.981843
1,余德龍,樂天桃猿,1.000000,0.000000,6.200000,0,0,0,1,0,3,0,0,9.606541,1.096458
2,傅于剛,統一7-ELEVEn獅,11.666667,6.942857,5.600000,1,1,4,7,4,18,5,14,6.424623,1.002275
3,優瑪,富邦悍將,6.333333,5.684211,2.884211,0,0,4,2,2,8,2,12,2.163124,1.041675
4,克維斯,統一7-ELEVEn獅,61.000000,3.540984,3.544262,2,3,59,32,13,54,16,84,2.931594,1.032662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,21.333333,4.218750,3.387500,1,1,10,3,6,29,7,32,3.239529,1.063268
134,黃子鵬,樂天桃猿,158.666667,2.325630,3.244118,10,8,96,25,36,166,70,233,3.413535,1.033766
135,黃恩賜,中信兄弟,25.666667,4.558442,3.511688,2,1,12,5,6,26,13,54,3.512223,0.978773
136,黃東淯,味全龍,1.666667,5.400000,3.800000,0,0,1,1,0,6,1,1,5.958443,1.041728


# 五.計算WAR
## RPW 也需額外手動輸入 做場次校正 
### 輸入方式 中職RPW = RPW(使用FG) x 120/162 

In [45]:
#2023 聯盟tRA=3.4380636410471386
#2022 聯盟tRA=3.347000
#RAR＝（1.356×聯盟平均tRA－PF補正後tRA）÷9×（局數）
pitcher["RAR"]=((1.365*lg_tRA-pitcher["tRA"]*pitcher["PF(tRA)"])/9*pitcher["IP"])

#2023 聯盟RPW(FG)=9.35166     
#2023 聯盟RPW(1.02)=6.8593172473
#2022 聯盟RPW(FG)=9.102359373
#2022 聯盟RPW(1.02)=6.723292499

RPW = 9.35166 *120/162
pitcher["WAR(FG)"]=pitcher["RAR"]/RPW
pitcher["WAR(1.02)"]=pitcher["RAR"]/6.8593172473
pitcher

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18096\2582389014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitcher["RAR"]=((1.365*lg_tRA-pitcher["tRA"]*pitcher["PF(tRA)"])/9*pitcher["IP"])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18096\2582389014.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitcher["WAR(FG)"]=pitcher["RAR"]/RPW
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18096\2582389014.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,投手,球隊,IP,ERA,FIP,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA,PF(tRA),RAR,WAR(FG),WAR(1.02)
0,伍鐸,味全龍,83.000000,2.277108,3.175904,4,1,43,16,10,89,44,136,3.594014,0.981843,14.848538,2.202234,2.164725
1,余德龍,樂天桃猿,1.000000,0.000000,6.200000,0,0,0,1,0,3,0,0,9.606541,1.096458,-0.599370,-0.088894,-0.087380
2,傅于剛,統一7-ELEVEn獅,11.666667,6.942857,5.600000,1,1,4,7,4,18,5,14,6.424623,1.002275,-1.685700,-0.250012,-0.245753
3,優瑪,富邦悍將,6.333333,5.684211,2.884211,0,0,4,2,2,8,2,12,2.163124,1.041675,2.030585,0.301162,0.296033
4,克維斯,統一7-ELEVEn獅,61.000000,3.540984,3.544262,2,3,59,32,13,54,16,84,2.931594,1.032662,14.311245,2.122547,2.086395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,黃偉晟,樂天桃猿,21.333333,4.218750,3.387500,1,1,10,3,6,29,7,32,3.239529,1.063268,4.016246,0.595662,0.585517
134,黃子鵬,樂天桃猿,158.666667,2.325630,3.244118,10,8,96,25,36,166,70,233,3.413535,1.033766,28.384484,4.209793,4.138092
135,黃恩賜,中信兄弟,25.666667,4.558442,3.511688,2,1,12,5,6,26,13,54,3.512223,0.978773,4.851492,0.719540,0.707285
136,黃東淯,味全龍,1.666667,5.400000,3.800000,0,0,1,1,0,6,1,1,5.958443,1.041728,-0.197822,-0.029340,-0.028840


In [46]:
#pitcher.to_excel('2022WAR_PF2.0.xlsx', index=True)